# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [68]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [69]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [70]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'fruits_db', 'local', 'met', 'uk_food']

In [71]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [72]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [73]:
# review a document in the establishments collection
db['establishments'].find_one()

{'_id': ObjectId('64ebc7c5fd0dfbcbccf6e0a3'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00

In [74]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [75]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [76]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
insert_result.inserted_id  # Should return the ObjectID of the inserted document


ObjectId('64ed6d2d4efb4f393c44f803')

In [77]:
# Check that the new restaurant was inserted
establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('64ed50444efb4f393c44f7fd'),
 'FHRSID': 'NewID',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '123 Penang Street',
 'PostCode': 'P123NG',
 'RatingValue': 5,
 'LocalAuthorityName': 'London',
 'geocode': {'longitude': 0.0, 'latitude': 0.0}}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [78]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = ['BusinessTypeID', 'BusinessType']

establishments.find_one(query, fields)

{'_id': ObjectId('64ebc7c5fd0dfbcbccf6e0a3'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [79]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    new_restaurant, 
    {'$set': 
        {'BusinessTypeID': 1}
    }
)

In [80]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('64ed50444efb4f393c44f7fd'),
 'FHRSID': 'NewID',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '123 Penang Street',
 'PostCode': 'P123NG',
 'RatingValue': 5,
 'LocalAuthorityName': 'London',
 'geocode': {'longitude': 0.0, 'latitude': 0.0}}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [81]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [82]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [83]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({'LocalAuthorityName': 'Dover'}))

0


In [84]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('64ebc7c5fd0dfbcbccf6e0a3'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [85]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 
                                         'geocode.latitude': {'$toDouble': '$geocode.latitude'}
                                         }
                                }
                               ]
                          )

Use `update_many` to convert `RatingValue` to integer numbers.

In [86]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [87]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["Exempt", "Pass", "Fail", "Awaiting Inspection"]
establishments.update_many(
    {"RatingValue": {"$in": non_ratings}},
    {"$set": {"RatingValue": None}}
)

In [88]:
# Check that the coordinates and rating value are now numbers

# Query a sample document to check the data types
sample_doc = establishments.find_one({}, {"geocode": 1, "RatingValue": 1})

# Check the type of 'latitude' and 'longitude'
latitude_type = type(sample_doc['geocode']['latitude'])
longitude_type = type(sample_doc['geocode']['longitude'])

# Check the type of 'RatingValue'
rating_value_type = type(sample_doc['RatingValue'])

latitude_type, longitude_type, rating_value_type


(float, float, int)